In [3]:
import pandas  as pd
import sqlite3 as db
import requests
import json

In [27]:
flights_url = "https://data.gov.il/api/3/action/datastore_search?resource_id=e83f763b-b7d7-479e-b172-ae981ddc6de5&limit=1000"
flights_api_response = requests.get(flights_url).json()
pd_flights = pd.json_normalize(flights_api_response['result']['records'])
pd_flights

,_id,CHOPER,CHFLTN,CHOPERD,CHSTOL,CHPTOL,CHAORD,CHLOC1,CHLOC1D,CHLOC1TH,CHLOC1T,CHLOC1CH,CHLOCCT,CHTERM,CHCINT,CHCKZN,CHRMINE,CHRMINH
0,1,PC,796,PEGASUS AIRLINES,2022-07-25T11:10:00,2022-07-25T11:31:00,D,SAW,ISTANBUL-SABIHA GOKCHEN,איסטנבול,ISTANBUL,טורקיה,TURKEY,3,42-45,B,DEPARTED,המריאה
1,2,TWI,503,TAILWIND AIRLINES,2022-07-25T11:10:00,2022-07-25T11:33:00,A,AYT,ANTALYA,אנטליה,ANTALYA,טורקיה,TURKEY,3,,,LANDED,נחתה
2,3,W6,8288,WIZZAIR,2022-07-25T10:55:00,2022-07-25T11:33:00,D,FCO,ROME - FIUMICINO,רומא,ROME,איטליה,ITALY,1,301-310,A,DEPARTED,המריאה
3,4,251,BDK06,תעשיה אווירית- נסויי טיסה,2022-07-25T11:30:00,2022-07-25T11:37:00,D,TLV,TEL AVIV,נתבג,BEN GURION,ישראל,ISRAEL,3,,,DEPARTED,המריאה
4,5,FR,4103,RYANAIR,2022-07-25T11:35:00,2022-07-25T11:41:00,A,PFO,PAPHOS,פאפוס,PAPHOS,קפריסין,CYPRUS,3,,,LANDED,נחתה
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,AHS,RA027,AERO HANDLING,2022-07-26T22:30:00,2022-07-26T22:30:00,A,VKO,MOSCOW-VNUKOVO,מוסקבה,MOSCOW,רוסיה,RUSSIAN FEDERATION,3,,,CANCELED,מבוטלת
996,997,PC,785,PEGASUS AIRLINES,2022-07-26T22:30:00,2022-07-26T22:30:00,A,SAW,ISTANBUL-SABIHA GOKCHEN,איסטנבול,ISTANBUL,טורקיה,TURKEY,3,,,NOT FINAL,לא סופי
997,998,U8,102,TUS AIRWAYS,2022-07-26T21:45:00,2022-07-26T22:32:00,A,LCA,LARNACA,לרנקה,LARNACA,קפריסין,CYPRUS,3,,,DELAYED,עיכוב
998,999,5W,7086,WIZZ AIR ABU DHABI,2022-07-26T22:35:00,2022-07-26T22:35:00,D,AUH,ABU DHABI,אבו דאבי,ABU DHABI,איחוד האמירויות הערב,UNITED ARAB EMIRATES,3,56-61,C,ON TIME,בזמן


In [28]:
countries_url = "https://www.geonames.org/countries/"
pd_countries = pd.read_html(countries_url)[1]
pd_countries

,ISO-3166alpha2,ISO-3166alpha3,ISO-3166numeric,fips,Country,Capital,Area in km²,Population,Continent
0,AD,AND,20,AN,Andorra,Andorra la Vella,468.0,77006,EU
1,AE,ARE,784,AE,United Arab Emirates,Abu Dhabi,82880.0,9630959,AS
2,AF,AFG,4,AF,Afghanistan,Kabul,647500.0,37172386,AS
3,AG,ATG,28,AC,Antigua and Barbuda,St. John's,443.0,96286,NaN
4,AI,AIA,660,AV,Anguilla,The Valley,102.0,13254,NaN
...,...,...,...,...,...,...,...,...,...
245,YE,YEM,887,YM,Yemen,Sanaa,527970.0,28498687,AS
246,YT,MYT,175,MF,Mayotte,Mamoudzou,374.0,279471,AF
247,ZA,ZAF,710,SF,South Africa,Pretoria,1219912.0,57779622,AF
248,ZM,ZMB,894,ZA,Zambia,Lusaka,752614.0,17351822,AF


In [32]:
con = db.connect("aiola.db")
pd_countries.to_sql(name="countries", con=con, if_exists='replace')
pd_flights.to_sql(name="flights", con=con, if_exists='replace')

1000

In [52]:
q1_query = """
    SELECT CHLOCCT as country, count(*) as flights 
    FROM flights 
    where CHRMINE == 'ON TIME' 
    group by 1 
    order by 2 desc 
    limit 3
"""
q1 = pd.read_sql_query(q1_query, con)
q1

,country,flights
0,TURKEY,21
1,ITALY,16
2,GERMANY,10


In [62]:
q2_query = """
    SELECT c.country, count(*) as landed
    FROM flights f
    INNER JOIN countries c
    ON f.CHLOCCT = UPPER(c.country)
    AND f.CHRMINE == 'LANDED'
    AND `Area in km²` > 1000000
    group by 1 
"""
q2 = pd.read_sql_query(q2_query, con)
q2

,Country,landed
0,Canada,2
1,Egypt,4
2,Ethiopia,2
3,Kazakhstan,1
4,United States,16


In [66]:
q3_query = """ 
    WITH count_flights AS (
        SELECT CHLOCCT as country, count(*) as cnt FROM flights 
        GROUP BY 1
    )
    SELECT f.country, f.cnt 
    FROM count_flights f
    INNER JOIN countries c
    ON f.country = UPPER(c.country)
    AND c.population > 10000000
    
"""
q3 = pd.read_sql_query(q3_query, con)
q3

,country,cnt
0,BELGIUM,17
1,CANADA,7
2,CHINA,5
3,GERMANY,75
4,EGYPT,11
5,SPAIN,53
6,ETHIOPIA,4
7,FRANCE,59
8,UNITED KINGDOM,43
9,GREECE,95


In [63]:
withq = """
 with temp as (
    SELECT * from countries
 )
 select * from temp
"""

rr = pd.read_sql_query(withq, con)
rr

,index,ISO-3166alpha2,ISO-3166alpha3,ISO-3166numeric,fips,Country,Capital,Area in km²,Population,Continent
0,0,AD,AND,20,AN,Andorra,Andorra la Vella,468.0,77006,EU
1,1,AE,ARE,784,AE,United Arab Emirates,Abu Dhabi,82880.0,9630959,AS
2,2,AF,AFG,4,AF,Afghanistan,Kabul,647500.0,37172386,AS
3,3,AG,ATG,28,AC,Antigua and Barbuda,St. John's,443.0,96286,None
4,4,AI,AIA,660,AV,Anguilla,The Valley,102.0,13254,None
...,...,...,...,...,...,...,...,...,...,...
245,245,YE,YEM,887,YM,Yemen,Sanaa,527970.0,28498687,AS
246,246,YT,MYT,175,MF,Mayotte,Mamoudzou,374.0,279471,AF
247,247,ZA,ZAF,710,SF,South Africa,Pretoria,1219912.0,57779622,AF
248,248,ZM,ZMB,894,ZA,Zambia,Lusaka,752614.0,17351822,AF


In [61]:
qq = pd.read_sql_query("SELECT `Area in km²` FROM countries", con)
qq

,Area in km²
0,468.0
1,82880.0
2,647500.0
3,443.0
4,102.0
...,...
245,527970.0
246,374.0
247,1219912.0
248,752614.0
